In [2]:
import requests 
import pandas as pd
import time
import json
from datetime import date,datetime,timedelta

In [3]:
#Declare file config
file_path = '../credentials/config.json'

#Open and read Json file
with open(file_path, "r") as file:
    config_data = json.load(file)

In [4]:
current_date = date.today()
current_date = current_date.strftime('%d/%m/%Y')
print("Current Date:", current_date)


Current Date: 20/10/2025


In [5]:
url = "https://fc-data.ssi.com.vn/api/v2/Market/AccessToken"

payload = {
    "consumerID": config_data["consumerID"],
    "consumerSecret": config_data["consumerSecret"]
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)
if response.status_code == 200:
    access_token = response.json()['data'].get("accessToken")
else:
    print("Failed to get access token. Status Code:", response.status_code)
    print("Response Body:", response.text)
    exit()

In [20]:
start_date = (date.today() - timedelta(days = 0)).strftime('%d/%m/%Y')
end_date = (date.today() - timedelta(days = 0)).strftime('%d/%m/%Y')

In [21]:
print(start_date)
print(end_date)

20/10/2025
20/10/2025


In [8]:
# Replace with the token you obtained
url = "https://fc-data.ssi.com.vn/api/v2/Market/IndexList"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

params = {
    "pageIndex": 1,
    "pageSize": 1000
}

# Send the GET request
response = requests.get(url, headers=headers,params=params)

# Extract data from the response JSON
data = response.json().get('data', [])

index_list = []
for i in range(len(data)):
    index_list.append(data[i]['IndexCode'])

print(index_list)

['HNX30', 'HNXIndex', 'HNXUpcomIndex', 'VN100', 'VN30', 'VNALLSHARE', 'VNCOND', 'VNCONS', 'VNDIAMOND', 'VNENE', 'VNFIN', 'VNFINLEAD', 'VNFINSELECT', 'VNHEAL', 'VNIND', 'VNINDEX', 'VNIT', 'VNMAT', 'VNMIDCAP', 'VNREAL', 'VNSI', 'VNSMALLCAP', 'VNUTI', 'VNX50', 'VNXALLSHARE', None]


In [22]:
# Replace with the token you obtained
url = "https://fc-data.ssi.com.vn/api/v2/Market/DailyIndex"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

df = pd.DataFrame()

for index in index_list:
    params = {
        "indexId": index,
        "fromDate":start_date,
        "toDate": end_date,
        "pageIndex": 1,
        "pageSize": 1000,
        "ascending": False
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json().get('data', [])
    df_response = pd.DataFrame(data)
    df = pd.concat([df, df_response], ignore_index=True)
    time.sleep(1)
    if len(df_response) == 0:
        break

In [23]:
display(df)

,IndexId,IndexValue,TradingDate,Time,Change,RatioChange,TotalTrade,TotalMatchVol,TotalMatchVal,TypeIndex,...,Advances,NoChanges,Declines,Ceilings,Floors,TotalDealVol,TotalDealVal,TotalVol,TotalVal,TradingSession
0,HNX30,563.19,20/10/2025,None,-44.94999999999990,-7.39,0,134113928,3755553497000,None,...,3,1,19,0,7,18461196,340639909200,152575124,4096193406200,C
1,HNXIndex,263.02,20/10/2025,None,-13.09,-4.74,0,168904693,4182233820600,None,...,25,38,108,3,34,21374436,376527709200,190279129,4558761529800,C
2,HNXUpcomIndex,109,20/10/2025,None,-3.67,-3.26,0,58635581,929501280500,None,...,63,62,178,7,7,4929314,74087558300,63564895,1003588838800,C
3,VN100,1800.51,20/10/2025,None,-1.0789,-5.65,0,1388105369,46664398442050,None,...,0,0,0,0,52,58021402,2347874640150,1446126771,49012273082200,C
4,VN30,1870.86,20/10/2025,None,-1.0628,-5.38,0,777472239,28714336249900,None,...,0,0,0,0,13,36374948,1570706702950,813847187,30285042952850,C
5,VNALLSHARE,1783.39,20/10/2025,None,-1.0575,-5.60,0,1533838698,48851957627140,None,...,0,0,0,3,91,64871893,2444515983150,1598710591,51296473610290,C
6,VNCOND,2213.25,20/10/2025,None,-1.1664,-5.01,0,30841938,1686232773590,None,...,0,0,0,0,5,5161764,283319580000,36003702,1969552353590,C
7,VNCONS,691.72,20/10/2025,None,-0.4217,-5.75,0,78117885,4065922067170,None,...,0,0,0,0,5,5047000,182792700000,83164885,4248714767170,C
8,VNDIAMOND,2543.88,20/10/2025,None,-1.4770,-5.49,0,376847393,12875876488150,None,...,0,0,0,0,10,25300130,987150036600,402147523,13863026524750,C
9,VNENE,511.22,20/10/2025,None,-0.2117,-3.98,0,11240429,240813901650,None,...,0,0,0,0,0,20000,409880000,11260429,241223781650,C
